TODO for competition exercises:

For Exercise 1:
- Make a function to determine bottleneck size
- Make the net more deep, and add a fc layer with bottlnek_size
- Add an early stoping parameter
- Use Tensorboard to visualize the train steps

For Exercise 2:

For Optional A:

For Optional B:
1.   Improve architecture
2.   Add a conditional factor to choose the number to generate:
- Add an embedding of a one hot encoding of the 10 numbers in the input of the generator and discriminator.




# Optional B: Generative Network

In this section is designed a vanilla implementation of a Generative Adversarial Network for MNIST number generation.

In this case we also add the possibility to conditionate the network with the value to generate.


In [0]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
import random
import logging

from torchvision import datasets, transforms
from timeit import default_timer as timer
from torch.autograd.variable import Variable

*Get training dataset, convert to tensors and normalize*

In [0]:
BATCH_SIZE = 100

In [4]:
mnist_trainset = datasets.MNIST('data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,)) # normalize to mean and std deviation of the MNIST dataset
                                ]))


0it [00:00, ?it/s]

9920512it [00:06, 1604933.72it/s]                            


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 130508.19it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


1654784it [00:00, 2362198.10it/s]                           
0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


8192it [00:00, 49996.85it/s]            

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [0]:
train_loader = torch.utils.data.DataLoader(
    mnist_trainset,
    batch_size=BATCH_SIZE, 
    shuffle=True)

num_batches = len(train_loader)

## Definition of the Generator and Discriminator classes

** Provar altres paràmetres:
- Provar amb LeakyReLU
- Afegir dropout


In [0]:
# Generator class
class Generator (nn.Module):
    def __init__(self,inp,out):
        super(Generator,self).__init__()

        self.label_embedding = nn.Embedding(10,10) #num_embedings, embedding_dim (one-hot vector)

        self.gen = nn.Sequential(
              nn.Linear(inp+10, 256),     #inp = The input of the generator is a lattent vector of 'X' size
              nn.LeakyReLU(0.2), 
              nn.Linear(256,512),
              nn.LeakyReLU(0.2),
              nn.Linear(512,1024),
              nn.LeakyReLU(0.2),
              nn.Linear(1024,out),    #out = The output of the generator has to be a vector of 28*28 = 784
              nn.Tanh()
          )

    def forward(self,x, labels):
        label = self.label_embedding(labels)
        out = torch.cat([x,label], 1)         #Concatenate the image vector x with his respective label one-hot encoded
        out = self.gen(out)
        return out


In [0]:
# Discriminator class
class Discriminator (nn.Module):
    def __init__(self,inp,out):
        super(Discriminator,self).__init__()

        self.label_embedding = nn.Embedding(10,10) #num_embedings, embedding_dim (one-hot vector)

        self.discrim = nn.Sequential(
            nn.Linear(inp+10, 1024),     #inp = The input of the dicriminator is a vector of image input (28*28)
            nn.LeakyReLU(0.2, inplace = True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Dropout(0.3),
            nn.Linear(256, out),      #out = This has to be 1 value with prob of Real or Fake
            nn.Sigmoid(),             #This activation function will be used to set 0 (fake) or 1 (real)
        )

    def forward(self,x, labels):
        label = self.label_embedding(labels)
        out = torch.cat([x,label], 1)         #Concatenate the image vector x with his respective label one-hot encoded
        out = self.discrim(out)
        return out


## Training functions

We first will define the following hiperparameters:
 - Loss: In this case we will Binary Cross Entropy
 - Optimizers: Stochastic Gradient dicent with their respective parameters

In [0]:

input_size_gen = 100
input_size_dis = 784
output_size_gen = 784
output_size_dis = 1

hparams = {
    'train_batch_size':BATCH_SIZE,
    'num_epochs':200,
    'learning_rate':1e-4
}

# we select to work on GPU if it is available in the machine, otherwise we will run on cpu
hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.is_available():
  torch.cuda.random_seed(1)

# We initializate the gen and disc
generator = Generator(input_size_gen, output_size_gen)
discriminator = Discriminator(input_size_dis, output_size_dis)

# Loss for Generator and Discriminator
loss = nn.BCELoss()

# Optimizers
gen_optimizer   = optim.Adam(generator.parameters(), lr = hparams['learning_rate'])
dis_optimizer   = optim.Adam(discriminator.parameters(), lr = hparams['learning_rate'])

The following function will be used to plot the generated image while training

In [0]:
def plot_gen_img(array_img, num = None):
  array_img   = array_img.detach()
  array_img = array_img.squeeze(0)
  array_img = array_img.squeeze(0)
  plt.imshow(array_img, cmap = 'binary')
  plt.xticks([])
  plt.yticks([])
  if num:
      plt.xlabel(num,fontsize='x-large')
  plt.show()



The following functions will be used to transform the images to vectors or vice versa.

In [0]:
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)

We also need to be able to create some random nosie to introduce to the Generative model with the following function:

In [0]:
def create_noise(size):
    n = Variable(torch.randn(size, 100))
    return n

We create 'X' random vectors to create a test input to send it through the generator and check the synthetized output every 'X' batches.

In [0]:
num_test_samples = 1
test_noise = create_noise(num_test_samples)

Functions to create ground truth labels when real_images or fake_images

- Vector of 1's when Real Images
- Vector of 0's when Fake Images

In [0]:
def real_target(size):
    data = Variable(torch.ones(size, 1)) 
    return data

def fake_target(size):
    data = Variable(torch.zeros(size, 1))
    return data

def fake_labels(size):
    data = Variable(torch.LongTensor(np.random.randint(0,10,size))) # Tensor filled with random int generated between low and high
    return data

def create_label_num(size):
    data = Variable(torch.LongTensor(np.arange(size)))
    return data

In [0]:
def train_discriminator(real_inp, labels, fake_inp, fake_labels, dis_opt, N):
    discriminator.train()
    device = hparams['device']
    # Reset Gradients
    dis_opt.zero_grad()

    # Train Discriminator with real images
    prediction_real  = discriminator.forward(real_inp, labels)          # Send the input through the discriminator model
    dis_real_loss = loss(prediction_real, real_target(N))       # As a ground truth we set a vector of 1 (Real)
    dis_real_loss.backward()

    # Train Discriminator with the image produced by the Generator
    prediction_fake  = discriminator(fake_inp, fake_labels)
    dis_fake_loss = loss(prediction_fake, fake_target(N)) # As a ground truth we set a vector of 0 (Fake)
    dis_fake_loss.backward()

    #We Optimize the Discriminator
    dis_opt.step()
    return dis_real_loss + dis_fake_loss , prediction_real, prediction_fake      

In [0]:
def train_generator(fake_inp, gen_opt, N):
      generator.train()
      device = hparams['device']
      # Reset Gradients
      gen_opt.zero_grad()
      
      #Create FAKE labels
      fake_lbls = fake_labels(N)
      # Send the random input through the Generator
      prediction = discriminator(fake_inp, fake_lbls)

      # Compute Loss for the Generator
      gen_train_loss = loss(prediction, real_target(N)) # As a ground truth we set a vector of 1 (Real) to make the Generator learn
      gen_train_loss.backward()


      gen_opt.step()
      return gen_train_loss

In [0]:
def train_net(train_loader, gen_opt, dis_opt, num_epochs):
    gen_losses = []
    dis_losses = []
    generator.to(hparams['device']) 
    discriminator.to(hparams['device'])
    # train over the number of epochs
    for epoch in range(1,num_epochs + 1):
        for batch_idx, (real_inp, labels) in enumerate(train_loader):
            N = real_inp.size(0) # Batch size == BATCH_SIZE defined above

            #Call function to create REAL data vector
            real_data = Variable(images_to_vectors(real_inp))
            #REAL labels
            real_labels = Variable(labels)
            #Call function to create FAKE labels
            fake_lbls = fake_labels(N)
            #Call function to create FAKE data vector
            fake_data = generator(create_noise(N), fake_lbls).detach()    #The detach() method constructs a new view on a tensor which is declared not to need gradients

            # Train Discriminator
            disc_loss, d_pred_real, d_pred_fake = train_discriminator(real_data, real_labels, fake_data, fake_lbls, dis_opt, N)

            #Forward a random vector through the Generator to obtain the synthetized output
            fake_data = generator(create_noise(N),fake_lbls)

            # Train Generator
            gen_loss = train_generator(fake_data, gen_opt, N)


            # Print losses for each epoch
            if (batch_idx) % 100 == 0:
                print(f'Epoch {epoch}: Batch {batch_idx} Discrim loss {disc_loss}, Generator loss {gen_loss}')
                gen_losses.append(gen_loss)
                dis_losses.append(disc_loss)
                losses = {'gen_losses':gen_losses, 'dis_losses':dis_losses}
                label_to_display = create_label_num(num_test_samples)
                test_images = vectors_to_images(generator(test_noise,label_to_display))
                plot_gen_img(test_images)

    # Print Graphic of gen and discr losses   
    plt.figure(figsize=(10, 8))
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.plot(gen_losses, label='GEN')
    plt.plot(dis_losses, label='DISC')
    plt.legend()

Train model

In [0]:
train_net(train_loader, gen_optimizer, dis_optimizer, hparams['num_epochs'])

Output hidden; open in https://colab.research.google.com to view.

In this section we will test the generator printing all the numbers

In [0]:
noise_vector = Variable(torch.randn(100, 100))
labels = Variable(torch.LongTensor([i for _ in range(10) for i in range(10)]))
num_images = vectors_to_images(generator(noise_vector, labels))
grid = make_grid(num_images, nrow=10, normalize=True).permute(1,2,0).numpy()
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(grid)
_ = plt.yticks([])
_ = plt.xticks(np.arange(15, 300, 30), ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], fontsize=20)

In this section we will be able select the number and generate it: 

In [0]:
# Select number to generate:
NUM_TO_GENERATE = 3
random_noise = create_noise(1) # Noise vector to send it through the generator

num_image = vectors_to_images(generator(random_noise,NUM_TO_GENERATE))
plot_gen_img(num_image)

-----------------------------------------------------------------------